In [1]:
import os 
import csv

import influxdb_client
from influxdb_client.client.write_api import ASYNCHRONOUS
from influxdb_client.client.exceptions import InfluxDBError
from urllib3.exceptions import NewConnectionError


data = os.listdir("./data/crypto_files")

INFLUX_URL = "http://influxdb2:8086"
INFLUX_TOKEN = "MyInitialAdminToken0=hola"

client = None
try:
    client = influxdb_client.InfluxDBClient(
        url=INFLUX_URL,
        token=INFLUX_TOKEN,
        org="docs"
    )

    print(f"Verificando estado de salud de InfluxDB en {INFLUX_URL}...")
    health = client.health()

    if health.status == "pass":
        print("Conexión exitosa")
    else:
        print("Conexión fallida")

except(InfluxDBError,NewConnectionError) as e:
    print("Error")
    print(e)

Verificando estado de salud de InfluxDB en http://influxdb2:8086...
Conexión exitosa


In [2]:
from datetime import datetime,timezone
from influxdb_client import Point, WriteOptions

writeoptions = WriteOptions(
        batch_size = 5000,
        flush_interval = 1000,
        write_type = "ASYNCHRONOUS"
    )

writre_api = client.write_api(write_options=writeoptions)

cont = 0

for nombre in data:

    
    with open("./data/crypto_files/"+nombre, 'r', encoding='utf-8') as f:
        
        lector = csv.reader(f)
        
        
        next(lector) 

        for row in lector:
            index = row[0]
            name = row[1]
            Symbol = row[2]
            
            date_ = row[3]
            dt = datetime.strptime(date_,"%Y-%m-%d %H:%M:%S")
            dt_utc = dt.replace(tzinfo=timezone.utc)
            time_date = dt_utc.isoformat().replace('+00:00','Z')
                
            high = float(row[4])
            low = float(row[5])
            open_ = float(row[6])
            close = float(row[7])
            volume = float(row[8])
            marketcap = float(row[9])

            p = Point("Criptos").tag("symbol",Symbol).tag("name",name).time(time_date) \
                .field("Close",close).field("high",high).field("low",low).field("open", open_) \
                .field("volume",volume).field("marketcap",marketcap)
            
            writre_api.write(bucket="crypto_raw" , org="docs" , record=p)
            cont = cont + 1
         
        print(f"Fichero {f} procesado")        
        

writre_api.close()
print(cont)

Fichero <_io.TextIOWrapper name='./data/crypto_files/coin_Aave.csv' mode='r' encoding='utf-8'> procesado
Fichero <_io.TextIOWrapper name='./data/crypto_files/coin_BinanceCoin.csv' mode='r' encoding='utf-8'> procesado
Fichero <_io.TextIOWrapper name='./data/crypto_files/coin_Bitcoin.csv' mode='r' encoding='utf-8'> procesado
Fichero <_io.TextIOWrapper name='./data/crypto_files/coin_Cardano.csv' mode='r' encoding='utf-8'> procesado
Fichero <_io.TextIOWrapper name='./data/crypto_files/coin_ChainLink.csv' mode='r' encoding='utf-8'> procesado
Fichero <_io.TextIOWrapper name='./data/crypto_files/coin_Cosmos.csv' mode='r' encoding='utf-8'> procesado
Fichero <_io.TextIOWrapper name='./data/crypto_files/coin_CryptocomCoin.csv' mode='r' encoding='utf-8'> procesado
Fichero <_io.TextIOWrapper name='./data/crypto_files/coin_Dogecoin.csv' mode='r' encoding='utf-8'> procesado
Fichero <_io.TextIOWrapper name='./data/crypto_files/coin_EOS.csv' mode='r' encoding='utf-8'> procesado
Fichero <_io.TextIOWrap